In [1]:
from typing_extensions import Literal, Annotated 
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph,START,END
from langgraph.types import Command
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
import os

In [2]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
LANGCHAIN_TRACING = os.getenv("LANGCHAIN_TRACING")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

os.environ["LANGCHAIN_TRACING"] = LANGCHAIN_TRACING
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [5]:
llm2 = ChatGroq(model_name = "deepseek-r1-distill-llama-70b")

Agent 1 will be the researcher agent and Agent 2 will be the analyst agent.

We will pass the input to the first agent and it will search the internet for related information and then it will pass the information to the analyst agent and it will interpret the results using a python code.

Both the agents are ReAct agents. 

There will be a loop between both the agents and the loop will continue until we have a perfect final answer. 

In [6]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [7]:
search_tool.invoke("Who is Ronaldo?")

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


"Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu] ⓘ; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for, and … Sep 17, 2025 · Cristiano Ronaldo is a Portuguese football sensation and global icon who has mesmerized fans worldwide with his extraordinary skills, numerous accolades, and record … Jun 7, 2025 · Cristiano Ronaldo is one of the greatest footballers of all time. Born on February 5, 1985, in Madeira, Portugal, he is a global sports icon known for his incredible goal-scoring … Ronaldo has indicated that he is not yet ready to retire from international duty, so he will need to keep playing club football if he wants to remain Portugal's captain. The World Cup is one of... Aug 12, 2025 · Cristiano Ronaldo's journey from Madeira to football legend, explore his career, records, achievements, early life, and personal milestones."

In [8]:
repl = PythonREPL()

In [9]:
code = """ 
x = 5
y = x * 2
print(y)
"""

In [10]:
repl.run(code)

Python REPL can execute arbitrary code. Use with caution.


'10\n'

In [11]:
@tool 
def python_refl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """
    Use this tool to execute python code. If you want to see the output of a value, 
    you should print it out with 'print(...)'. This is visible to the user.
    """
    try: 
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute code with error: {repr(e)}"
    
    result_str = f"Successfully executed code. The output was:\n{result}"

    return (
        result_str + "\n\n If you have completed all the tasks, please respond with 'Final Answer: <your final answer>'"
    )

In [12]:
def make_system_prompt(instruction: str) -> str:
    return (
        "You are a highly intelligent and capable AI assistant who has the capability to collaborate with other agents to complete complex tasks."
        "Use the provided tools to progress towards the final answer."
        "If you are unable to fully answer the question, that is okay, another assistant with different tools will help you where you left off. Execute what you " 
        "can to make progress. If you or the other assistants have the final answer or deliverable, prefix your response with FINAL ANSWER: so the team knows where to stop."
        f"\n{instruction}"
    )